In [1]:


# Local embedding model paths (downloaded above)O
#embd_model_path = "/home/matrix4284/MY_GEN_AI_PROJECTS/EMBEDDINGS/llama_embed_model/nomic-embed-text-v1.f16.gguf"
# embd_model_path = "/Users/rlm/Desktop/Code/llama.cpp/models/nomic-embd/nomic-embed-text-v1.f16.gguf"
# embd_model_path = "/Users/rlm/Desktop/Code/llama.cpp/models/nomic-embd/nomic-embed-text-v1.5.f16.gguf"
# embd_model_path = "/Users/rlm/Desktop/Code/llama.cpp/models/nomic-embd/nomic-embed-text-v1.5.Q4_K_S.gguf"

In [2]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings


import pandas as pd
#from langchain_community.vectorstores import Chroma



import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

#from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
import pickle
from langchain_community.chat_models import ChatOllama

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
#from langchain_community.embeddings import LlamaCppEmbeddings
#from langchain_nomic.embeddings import NomicEmbeddings
import os




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# Ollama model name
local_llm = "llama-3.1"
llm = ChatOllama(model=local_llm, temperature=0)
#llm.invoke("what is the capital of India ?")


In [3]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'  
pdf_enrichment_output_file = 'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'

with open(pdf_enrichment_output_dir+pdf_enrichment_output_file, 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [11]:
for idx in range(0,len(document_dict_deserialized)):
    for i in document_dict_deserialized[idx]['entities']:
        print(i['entity'])

Docker
Kubernetes
Pods
Containers
Linux namespaces
Network namespace
UTS namespace
IPC namespace
PID namespace
Volume
IP address
Port space
Pods
Kubernetes
Flat inter-pod network
Pod
Container
IP address
NAT (Network Address Translation)
Worker nodes
LAN (Local Area Network)
Software-defined network
VM (Virtual Machine)
Process
App
Node
Container 1
Container 2
Pod
Container
Kubernetes
Volume
Sidecar container
Log rotator
Collector
Data processor
Communication adapter
Frontend process
Backend process
Frontend container
Backend container
Pod
Frontend pod
Backend pod
Kubernetes
REST API
kubectl
YAML
JSON
Pods
API object definitions
Kubernetes API reference documentation
kubectl get command
kubectl run command
Pod metadata
Pod specification
containers
volumes
terminationMessagePath
dnsPolicy
nodeName
restartPolicy
serviceAccount
terminationGracePeriodSeconds
volumes
containerID
image
lastState
name
ready
restartCount
state
hostIP
phase
podIP
startTime
Kubernetes API version
Metadata
Spec
P

In [9]:
def decider(state):
    """
    Decides if the user is just wanting summary or summary with perspectives 
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    query = state_dict["query"]
    
    prompt = PromptTemplate(
        template="You are a supervisor tasked with managing user requests in form of conversation"
        "Given a user request: \n\n {question} \n\n select the task type it should belong to"
        " The task type can be one among : summarizer or perspective-based-summarizer"
        "Only Return one of these two words i/e. summarizer or perspective-based-summarizer"
        "in the output and nothing else",
        input_variables=["question"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()
    

    
    #for d in documents:
    score = chain.invoke(
            {
                "question": question,
            }
        )
    #grade = score["score"]
    print(score)
    
    print("GRADE")
          
    print(score)
          
    if "perspective" in score.lower():
        print("---AGENT: Perspective Based Summarizer---")
        task='perspective-based-summarizer'
            #filtered_docs.append(documents)
            #print(documents)
    else:
        print("---AGENT: Summarizer---")
        task='summarizer'
    
    if query=="summarize":
        query_type = 1
    else:
        query_type = 0
    
    return {"keys": {"query_type": query_type}}


def deserialize(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    document = state_dict["input_doc_serialized"]
    path = state_dict["dir"]
    
    with open(path+document, 'rb') as handle:
        document_dict_deserialized = pickle.load(handle)
    
    
    return {"keys": {"document_deserialized": document_dict_deserialized}}



def page_summarizer(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    document_dict_deserialized = state_dict["document_deserialized"]
    doc_len=len(document_dict_deserialized)

    full_doc_sumary=[]
    doc_len=len(document_dict_deserialized)
    for i in range(0,doc_len):
        summary=document[i]["summary"]
        full_doc_sumary.append(summary)
    
    return {"keys": {"document_deserialized": document_dict_deserialized,"full_document_summary": full_doc_sumary}}




def collate_summaries(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    document_dict_deserialized = state_dict["document_deserialized"]

    full_doc_sumary=[]
    doc_len=len(document_dict_deserialized)
    for i in range(0,doc_len):
        summary=document[i]["summary"]
        full_doc_sumary.append(summary)
    
    return {"keys": {"document_deserialized": document_dict_deserialized,"full_document_summary": full_doc_sumary}}




def entity_list_formatter(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    document_dict_deserialized = state_dict["document_deserialized"]
    summary =  state_dict["full_document_summary"]

    ent_lst=[]
    
    for idx in range(0,len(document_dict_deserialized)):
        for i in document_dict_deserialized[idx]['entities']:
            ent_lst.append(entity['entity'])
    
    return {"keys": {"full_document_summary": full_doc_sumary,"entity_list":ent_lst}}

In [ ]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'  
pdf_enrichment_output_file = 'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'

with open(pdf_enrichment_output_dir+pdf_enrichment_output_file, 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [5]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("decider", decider)
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("parse", parse)  # retrieve
workflow.add_node("filter_documents", filter_documents)  # grade documents
workflow.add_node("transform_query", transform_query)  # transform_query

workflow.add_node("summarizer", SummarizerCrew().kickoff)

workflow.add_node("trendanalyze", TrendAnalyzerCrew().kickoff)

#workflow.add_node("generate", generate)  # generatae
#workflow.add_node("transform_query", transform_query)  # transform_query
#workflow.add_node("prepare_for_final_grade", prepare_for_final_grade)  # passthrough

# Build graph
workflow.set_entry_point("decider")

workflow.add_edge("decider","retrieve")

workflow.add_edge("retrieve", "parse")

workflow.add_edge("parse", "filter_documents")

workflow.add_edge("transform_query", "retrieve")

workflow.add_conditional_edges(
    "filter_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "summarizer": "summarizer",
        "trendanalyze":"trendanalyze"
        
    },
)

workflow.add_conditional_edges(
    "summarizer",
    grade_generation_summary,
    {
        "useful": END,
        "not useful": "summarizer",
    },
)

workflow.add_conditional_edges(
    "trendanalyze",
    grade_generation_trend,
    {
        "useful": END,
        "not useful": "trendanalyze",
    },
)


# Compile
app = workflow.compile()

'A pod of containers allows you to run closely related processes together, providing them with almost the same environment as a single container, while keeping them somewhat isolated. Containers within a pod share certain resources like network interfaces and IPC namespaces, but have fully isolated filesystems unless shared using a Kubernetes Volume concept.'

In [8]:
##Web Search And Return URL,Title,Answer,Description

from langchain.chains import RetrievalQAWithSourcesChain

# fetch source docs details:
def extract_source_docs_metadata(result):
    sources = result['source_documents']
    metadatas = []
    for source in sources:
        metadata = {
            "title": source.metadata.get("title"),
            "description": source.metadata.get("description"),
            "source": source.metadata.get("source"),
            "answer": result["answer"]
        }
        metadatas.append(metadata)
    return metadatas

# webretriver to retrive the urls
def fetch_pages(input):
    qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
        llm,
        retriever=web_search_retriver_func(search, vectorstore, llm),
        #retriever=web_search_retriver_func(search=search, llm=llm),
        return_source_documents=True
    )
    return extract_source_docs_metadata(qa_chain({"question": input}))

#pages = fetch_pages("what are some best llmops tools ?")

#for page in pages:
#    print(page)

In [9]:
#Web Page Scraping Program Returning Information Extracted From Web Page

from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

def scrape_with_playwright(urls, bs4=True, tags=["span"], log=False):
    loader = AsyncChromiumLoader(urls)
    docs = loader.load()
    if bs4:
        bs_transformer = BeautifulSoupTransformer()
        docs_transformed = bs_transformer.transform_documents(
            docs, tags_to_extract=tags
        )
        if log:
            with open("test.txt", "w") as f:
                f.write(str(docs_transformed))
    else:
        docs_transformed = docs

    return docs_transformed

#doc = scrape_with_playwright( 
#    ['https://www.ibm.com/topics/llmops'],
#    tags=["main", "a", "p", "pre", "title", "span"]
#)

#write_pickle(doc, 'scraped_doc.pkl')
#read_pickle('scraped_doc.pkl')

In [10]:
#Caller Code For CrewAI State Node
def parse_pages(meta_lst,tags):
    urls=[]
    for i in meta_lst:
        urls.append(i['source'])
    
    return scrape_with_playwright(urls,tags)
     
        
    

In [11]:
from crewai import Agent, Task, Crew

class ContentGenerationAgents():
        
    def summary_agent(self):
        return Agent(
                role='Principal Researcher',
                goal=
                'Carry out Informative and concise summaries based on the content you are working with',
                backstory=
                "You're a Principal Content Developer of a big Company who can precisely describe lengthy documents.",
                allow_delegation=False,
                llm=llm,
                max_iterations=1,
                verbose=True
        )

    def trend_analysis_agent(self):
        return Agent(
                role='Principal Researcher',
                goal=
                'Extract Main Trends in bullet points based on the content you are working with',
                backstory=
                "You're a Principal Content Developer at a big company and you need to do a research based on the web search results",
                allow_delegation=False,
                llm=llm,
                max_iterations=1,
                verbose=True
        )

    

In [12]:
from crewai import Task
from textwrap import dedent

class ContentGenerationTasks:
    def summary_task(self, agent, content):
        #print(content)
        return Task(
            agent=agent,
            description="Analyze and summarize the content below,"
                    "make sure to include the most relevant information in the summary,"
                    "return only the summary nothing else."
                    f"\n\nCONTENT\n----------\n{content}",
            expected_output="give a well summary of the content and try to keep it between 250-500 words.",
            verbose=True
        )

    def trend_analysis_task(self, agent ,content):
        return Task(
            description="Analyze and extract top 10 trends from the content below,"
                    "make sure to include the most relevant information in the trends,"
                    "return only the trends as a list of bullet points."
                    f"\n\nCONTENT\n----------\n{content}",
            expected_output='A bullet point list of the top 10 most important Trends',
            agent=agent,
            #context=[summary_task]
        )

In [13]:
class SummarizerCrew():    
    
    def __init__(self):
        
        agents = ContentGenerationAgents()
        self.summary_agent = agents.summary_agent()
        #self.trend_analysis_agent=agents.trend_analysis_agent()

    def kickoff(self, state):
        print("### Summarizing Search Results")
        
        state_dict = state["keys"]
        documents = state_dict["documents"]
        question = state_dict["question"]
        
        print('Filtered Documents')
        print(documents)
        
        tasks = ContentGenerationTasks()
        crew = Crew(
        agents=[self.summary_agent],
        tasks=[
                tasks.summary_task(self.summary_agent, state["keys"]["documents"])
                #tasks.trend_analysis_task(self.trend_analysis_agent)
            ],
            verbose=True
        )
        result = crew.kickoff()
        print("summary")
        print(result)
        #return {**state, "summary": result}
        return {"keys": {"documents": documents, "summary": result , "question":question}}

In [14]:
class TrendAnalyzerCrew():    
    
    def __init__(self):
        
        agents = ContentGenerationAgents()
        #self.summary_agent = agents.summary_agent()
        self.trend_analysis_agent=agents.trend_analysis_agent()

    def kickoff(self, state):
        print("### Extracting Trend Analysis from Search Results")
        
        state_dict = state["keys"]
        #summary = state_dict["summary"]
        question = state_dict["question"]
        documents = state_dict["documents"]
        
        #print('Summary')
        #print(summary)
        
        tasks = ContentGenerationTasks()
        crew = Crew(
        agents=[self.trend_analysis_agent],
        tasks=[
                tasks.trend_analysis_task(self.trend_analysis_agent, state["keys"]["documents"])
                #tasks.trend_analysis_task(self.trend_analysis_agent)
            ],
            verbose=True
        )
        result = crew.kickoff()
        print("Trends")
        print(result)
        #return {**state, "summary": result}
        return {"keys": {"documents": documents, "question":question , "trend": result}}

In [15]:
from typing import Dict, TypedDict

from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [16]:
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

### Nodes ###


def decider(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK FUNCTION TO BE CALLED---")
    state_dict = state["keys"]
    question = state_dict["question"]
    

    

    # Prompt
    prompt = PromptTemplate(
        template="You are a supervisor tasked with managing user requests in form of conversation"
        "Given a user request: \n\n {question} \n\n select the task type it should belong to"
        " The task type can be one among : summarizer, trendanalyzer"
        "Only Return one of these two words i/e. summarizer or trendanalyzer in the output and nothing else",
        input_variables=["question"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()
    

    
    #for d in documents:
    score = chain.invoke(
            {
                "question": question,
            }
        )
    #grade = score["score"]
    print(score)
    
    print("GRADE")
          
    print(score)
          
    if "summarizer" in score.lower():
        print("---AGENT: Summarizer---")
        task='summarizer'
            #filtered_docs.append(documents)
            #print(documents)
    else:
        print("---AGENT: TrendAnalyzer---")
        task='trendanalyzer'
        #continue

    return {"keys": {"question": question,"task":task}}





def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    task=state_dict["task"]
    
    #documents = retriever.get_relevant_documents(question)
    
    meta_lst = fetch_pages(question)
    
    return {"keys": {"question": question,"meta_lst": meta_lst,"task":task}}



### Nodes ###


def parse(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    
    tags=["main", "a", "p", "pre", "title", "span"]
    
    state_dict = state["keys"]
    question = state_dict["question"]
    meta_lst=state_dict["meta_lst"]
    task=state_dict["task"]
    #documents = retriever.get_relevant_documents(question)
    
    parsed_content_lst = parse_pages(meta_lst,tags)
    
    return {"keys": {"question": question,"parsed_content_lst": parsed_content_lst,"task":task}}


def filter_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["parsed_content_lst"]
    task=state_dict["task"]
    
    
    print('Filter Document Task')
    print(task)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        There should not be any words or sentences associated with the score. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["question","context"],
    )

    # Chain
    #chain = prompt | llm | JsonOutputParser()
    
    chain = prompt | llm | StrOutputParser()

    # Score
    filtered_docs = []
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        #grade = score["score"]
        if "yes" in score.lower():
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(documents)
            #print(documents)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    return {"keys": {"documents": filtered_docs, "question": question, "task":task}}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    task=state_dict["task"]

    # LLM
    #llm = ChatOllama(model=local_llm, temperature=0)
    
    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n 
        Look at the input and try to reason about the underlying sematic intent / meaning. \n 
        Here is the initial question:
        \n ------- \n
        {question} 
        \n ------- \n
        Formulate an improved question:""",
        input_variables=["question"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {"keys": {"documents": documents, "question": better_question, "task":task}}




In [17]:
#Decider Function

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    task=state_dict["task"]
    
    print('decide to generate TASK:')
    print(task)

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY---")
        return "transform_query"
    elif task=='summarizer':
        # We have relevant documents, so generate answer
        print("---TASK: summarizer---")
        
        #print(filtered_documents)
        
        return "summarizer"
    elif task=='trendanalyzer':
        # We have relevant documents, so generate answer
        print("---TASK: trenanalyzer---")
        
        #print(filtered_documents)
        
        return "trendanalyze"
    
def wait_next_run(self, state):
    print("## Waiting for 180 seconds")
    time.sleep(180)
    return state


def grade_generation_summary(state):
    """
    Determines whether the generation addresses the question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Binary decision
    """

    print("---GRADE GENERATION SUMMARY vs QUESTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    summary = state_dict["summary"]

    #llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
        Here is the answer:
        \n ------- \n
        {summary} 
        \n ------- \n
        Here is the question: {question}
        Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["summary", "question"],
    )

    # Prompt
    chain = prompt | llm | JsonOutputParser()
    score = chain.invoke({"summary": summary, "question": question})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: USEFUL---")
        return "useful"
    else:
        print("---DECISION: NOT USEFUL---")
        return "not useful"
    
    
def grade_generation_trend(state):
    """
    Determines whether the generation addresses the question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Binary decision
    """

    print("---GRADE GENERATION TREND vs QUESTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    #documents = state_dict["summary"]
    trend = state_dict["trend"]

    #llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
        Here is the answer:
        \n ------- \n
        {trend} 
        \n ------- \n
        Here is the question: {question}
        Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["trend", "question"],
    )

    # Prompt
    chain = prompt | llm | JsonOutputParser()
    score = chain.invoke({"trend": trend, "question": question})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: USEFUL---")
        return "useful"
    else:
        print("---DECISION: NOT USEFUL---")
        return "not useful"
    
def grade_generation_summary(state):
    """
    Determines whether the generation addresses the question.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Binary decision
    """

    print("---GRADE GENERATION SUMMARY vs QUESTION---")
    state_dict = state["keys"]
    question = state_dict["question"]
    summary = state_dict["summary"]
    #trend = state_dict["trend"]

    #llm = ChatOllama(model=local_llm, format="json", temperature=0)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
        Here is the answer:
        \n ------- \n
        {summary} 
        \n ------- \n
        Here is the question: {question}
        Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["summary", "question"],
    )

    # Prompt
    chain = prompt | llm | JsonOutputParser()
    score = chain.invoke({"summary": summary, "question": question})
    grade = score["score"]

    if grade == "yes":
        print("---DECISION: USEFUL---")
        return "useful"
    else:
        print("---DECISION: NOT USEFUL---")
        return "not useful"
    
def wait_next_run(state):
        print("## Waiting for 180 seconds")
        time.sleep(180)
        return state

In [18]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("decider", decider)
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("parse", parse)  # retrieve
workflow.add_node("filter_documents", filter_documents)  # grade documents
workflow.add_node("transform_query", transform_query)  # transform_query

workflow.add_node("summarizer", SummarizerCrew().kickoff)

workflow.add_node("trendanalyze", TrendAnalyzerCrew().kickoff)

#workflow.add_node("generate", generate)  # generatae
#workflow.add_node("transform_query", transform_query)  # transform_query
#workflow.add_node("prepare_for_final_grade", prepare_for_final_grade)  # passthrough

# Build graph
workflow.set_entry_point("decider")

workflow.add_edge("decider","retrieve")

workflow.add_edge("retrieve", "parse")

workflow.add_edge("parse", "filter_documents")

workflow.add_edge("transform_query", "retrieve")

workflow.add_conditional_edges(
    "filter_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "summarizer": "summarizer",
        "trendanalyze":"trendanalyze"
        
    },
)

workflow.add_conditional_edges(
    "summarizer",
    grade_generation_summary,
    {
        "useful": END,
        "not useful": "summarizer",
    },
)

workflow.add_conditional_edges(
    "trendanalyze",
    grade_generation_trend,
    {
        "useful": END,
        "not useful": "trendanalyze",
    },
)


# Compile
app = workflow.compile()

In [21]:
# Run
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
inputs = {"keys": {"input_doc": "A detailed summary of Verizon Smart Family"}}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation


---CHECK FUNCTION TO BE CALLED---
Based on the user request provided, the task type that this request belongs to is:

Summarizer
GRADE
Based on the user request provided, the task type that this request belongs to is:

Summarizer
---AGENT: Summarizer---
"Node 'decider':"
'\n---\n'
---RETRIEVE---


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.48s/it]


"Node 'retrieve':"
'\n---\n'
---RETRIEVE---
"Node 'parse':"
'\n---\n'
---CHECK RELEVANCE---
Filter Document Task
summarizer
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---


2024-03-25 20:59:55,274 - 126639274956352 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---GRADE: DOCUMENT RELEVANT---
"Node 'filter_documents':"
'\n---\n'
---DECIDE TO GENERATE---
decide to generate TASK:
summarizer
---TASK: summarizer---
### Summarizing Search Results
Filtered Documents
[[Document(page_content="Accessibility Resource Center Skip to main content  Sign in  0  Sign in  0 iPhone 15. Newphoria. Get it on us. Online Only. No trade-in req'd. Limited time offer. Buy (https://www.verizon.com/smartphones/apple-iphone-15/)  |   Details. (javascript:void(0);)   iPhone:    (/support/) Support  (/support/)        (/support/services-and-apps/) Services & Apps  (/support/services-and-apps/)        (/support/verizon-smart-family/) Verizon Smart Family  (/support/verizon-smart-family/)       Verizon Smart Family FAQs    Was this helpful?  Was this helpful?  *The Verizon Smart Family account can be used to manage all compatible devices on the same Verizon mobile account. *iPads aren't currently supported. Tablets must have a line on your Verizon mobile account to be manag

Verizon's Smart Family app is a valuable tool for parents who want to monitor and manage their teenagers' digital habits. The app provides a range of features that allow parents to keep track of their child's online activity, set limits and restrictions, and receive alerts when certain behaviors are detected.

One of the key benefits of Smart Family is its ability to provide real-time insights into your teen's driving habits and help ease the minds of parents as their kids are driving on their own for the first time. The app can confirm that teens are driving responsibly by viewing trip activity after a trip is complete, and it also provides insights on speed, hard braking, and sharp turns. Additionally, Smart Family can detect potential crashes and send safety alerts to parents.

The app is available for both iPhone and Android devices and can be downloaded onto Verizon devices from the Apple App Store or Google Play. To get the full benefit of all features, the Smart Family Companion

In [21]:
output

{'__end__': {'keys': {'documents': [[Document(page_content="Accessibility Resource Center Skip to main content  Sign in  0  Sign in  0 iPhone 15. Newphoria. Get it on us. Online Only. No trade-in req'd. Limited time offer. Buy (https://www.verizon.com/smartphones/apple-iphone-15/)  |   Details. (javascript:void(0);)   iPhone:    (/support/) Support  (/support/)        (/support/services-and-apps/) Services & Apps  (/support/services-and-apps/)        (/support/verizon-smart-family/) Verizon Smart Family  (/support/verizon-smart-family/)       Verizon Smart Family FAQs    Was this helpful?  Was this helpful?  *The Verizon Smart Family account can be used to manage all compatible devices on the same Verizon mobile account. *iPads aren't currently supported. Tablets must have a line on your Verizon mobile account to be managed. **Most features require that the Verizon Smart Family Companion app (https://www.verizon.com/support/how-to-use-verizon-smart-family/#companion) is installed on th

In [ ]:
Trend Analyzer:
Analyze and provide the recent trends of Verizon Smart Family

Summarizer:
    
A detailed summary of Verizon Smart Family